<span style="color:orange; font-weight:bold">Note: To answer questions based on text documents, we recommend the procedure in <a href="https://github.com/openai/openai-cookbook/blob/main/examples/Question_answering_using_embeddings.ipynb">Question Answering using Embeddings</a>. Some of the code below may rely on <a href="https://github.com/openai/openai-cookbook/tree/main/transition_guides_for_deprecated_API_endpoints">deprecated API endpoints</a>.</span>

# 3. Train a fine-tuning model specialized for Q&A
This notebook will utilize the dataset of context, question and answer pairs to additionally create adversarial questions and context pairs, where the question was not generated on that context. In those cases the model will be prompted to answer "No sufficient context for answering the question". We will also train a discriminator model, which predicts whether the question can be answered based on the context or not.

We will add hard adversarial examples as well, which will be based either on semantically similar sections, or neighbouring sections, originating from the same article.

In [72]:
import openai
import pandas as pd
import yaml
import numpy as np


with open("key.yaml") as f:
    k = yaml.load(f, Loader=yaml.loader.SafeLoader)
openai.api_key = k["OPENAI_API_KEY"]


df = pd.read_csv('olympics-data/olympics_qa.csv')

df.head()

,title,heading,content,tokens,context,questions,answers
0,2020 Summer Olympics,Summary,"The 2020 Summer Olympics, officially the Games...",621,2020 Summer Olympics\nSummary\n\nThe 2020 Summ...,1. When were the 2020 Summer Olympics held?\n2...,1. The 2020 Summer Olympics were held from 23 ...
1,2020 Summer Olympics,Host city selection,The International Olympic Committee (IOC) vote...,126,2020 Summer Olympics\nHost city selection\n\nT...,1. When was the host city for the 2020 Summer ...,1. The host city for the 2020 Summer Olympics ...
2,2020 Summer Olympics,Impact of the COVID-19 pandemic,"In January 2020, concerns were raised about th...",375,2020 Summer Olympics\nImpact of the COVID-19 p...,1. What concerns were raised about the potenti...,1. Concerns were raised about the potential im...
3,2020 Summer Olympics,Qualifying event cancellation and postponement,Concerns about the pandemic began to affect qu...,298,2020 Summer Olympics\nQualifying event cancell...,1. What were some of the concerns that affecte...,1. Some of the concerns that affected qualifyi...
4,2020 Summer Olympics,Effect on doping tests,Mandatory doping tests were being severely res...,163,2020 Summer Olympics\nEffect on doping tests\n...,1. What impact did the COVID-19 pandemic have ...,1. The COVID-19 pandemic severely restricted d...


In [73]:
df["answers"].iloc[0]

'1. The 2020 Summer Olympics were held from 23 July to 8 August 2021.\n2. The 2020 Summer Olympics were held in Tokyo, Japan.\n3. The 2020 Summer Olympics were postponed due to the global COVID-19 pandemic.\n4. The 2020 Summer Olympics had a total spending of over $20 billion.\n5. Japan has hosted the Olympic Games four times, including the 2020 Summer Olympics.\n6. Some of the new events introduced in the 2020 Summer Olympics were 3x3 basketball, freestyle BMX, mixed gender team events, madison cycling for men and women, and new sports such as baseball and softball, karate, sport climbing, surfing, and skateboarding.\n7. The United States topped the medal count in the 2020 Summer Olympics by both total golds (39) and total medals (113).\n8. China finished second in the medal count in the 2020 Summer Olympics, with 38 gold medals and 89 total medals.\n9. Host nation Japan finished third in the medal count in the 2020 Summer Olympics, setting a record for the most gold medals (27) and t

Split the sections into a training and testing set

In [5]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
len(train_df), len(test_df)

(80, 20)

we check that the separator we intend to use isn't present within the contexts

In [74]:
df.context.str.contains('->').sum()

0

## 3.1 Create the fine-tuning datasets for Q&A and discriminator models
The fine-tuning dataset is created in the following way. For every corresponding question, answer and context pair we create:
- Positive example: correct question, answer, context pair
- Negative examples:
  - random negative example, where the random context is paired with the question 
  - two hard negative examples
    - one originating from the same wikipedia article
    - another, which is most similar to the correct context

This process is noisy, as sometimes the question might be answerable given a different context, but on average we hope this won't affect the peformance too much.

We apply the same process of dataset creation for both the discriminator, and the Q&A answering model. We apply the process separately for the training and testing set, to ensure that the examples from the training set don't feature within the test set.

In [75]:
def get_embedding(text, model="text-embedding-ada-002"):
   text = text.replace("\n", " ")
   return openai.Embedding.create(input = [text], model=model)['data'][0]['embedding']

In [82]:
df_contexts = {"context":[], "question":[], "embedding": []}
for i, row in df.iterrows():
    for q in row["questions"].split("\n"):
        try:
            emb = get_embedding(q[2:].strip())
        except:
            continue
        df_contexts["context"].append(row["context"])
        df_contexts["question"].append(q)
        df_contexts["embedding"].append(emb)

In [83]:
df_contexts = pd.DataFrame(df_contexts)
df_contexts.head()

,context,question,embedding
0,2020 Summer Olympics\nSummary\n\nThe 2020 Summ...,1. When were the 2020 Summer Olympics held?,"[0.0020419505890458822, -0.02178940176963806, ..."
1,2020 Summer Olympics\nSummary\n\nThe 2020 Summ...,2. Where were the 2020 Summer Olympics held?,"[0.0045273941941559315, -0.023686664178967476,..."
2,2020 Summer Olympics\nSummary\n\nThe 2020 Summ...,3. Why were the 2020 Summer Olympics postponed?,"[0.013307560235261917, -0.027771182358264923, ..."
3,2020 Summer Olympics\nSummary\n\nThe 2020 Summ...,4. How much money was spent on the 2020 Summer...,"[0.015362625010311604, -0.021405257284641266, ..."
4,2020 Summer Olympics\nSummary\n\nThe 2020 Summ...,5. How many times has Japan hosted the Olympic...,"[0.015828732401132584, -0.010652724653482437, ..."


In [99]:
df_contexts.to_csv('olympics-data/questions_context_and_embeddings.csv', index=False)

In [95]:
import random
def get_random_similar_contexts(question_emb, context, max_rerank=10):
    """
    Find similar contexts to the given context using the search file
    """
    distances = openai.embeddings_utils.distances_from_embeddings(question_emb, np.vstack(df_contexts['embedding']), distance_metric="cosine")
    neighbours = openai.embeddings_utils.indices_of_nearest_neighbors_from_distances(distances)[:max_rerank]
    candidates = []
    for i in neighbours:
        if df_contexts["context"].iloc[i] == context:
            continue
        candidates.append(df_contexts["context"].iloc[i])
    try:
        random_candidate = random.choice(candidates)
        return random_candidate
    except:
        return ""

In [96]:
get_random_similar_contexts(df_contexts.iloc[0,2], df_contexts.iloc[0,0])

'2020 Summer Olympics\nSports\n\nThe event program for the 2020 Summer Olympics was approved by the IOC executive board on 9 June 2017. IOC president Thomas Bach stated that their goal was to give the Games "youthful" and "urban" appeal, and to increase the number of female participants.The Games featured 339 events in 33 different sports, encompassing a total of 50 disciplines. Karate, sport climbing, surfing, and skateboarding made their Olympic debut, while baseball and softball also made a one-off return to the Summer Olympics for the first time since 2008. 15 new events within existing sports were also added, including 3×3 basketball, freestyle BMX, and the return of madison cycling, as well as 9 new mixed events in several sports (table tennis, archery, judo, shooting (3), triathlon, 4 × 400 m relay running and 4 × 100 m medley swimming).In the list below, the number of events in each discipline is noted in parentheses.'

In [97]:
def create_fine_tuning_dataset(df, discriminator=False, n_negative=1, add_related=False):
    """
    Create a dataset for fine tuning the OpenAI model; either for a discriminator model, 
    or a model specializing in Q&A, where it says if no relevant context is found.

    Parameters
    ----------
    df: pd.DataFrame
        The dataframe containing the question, answer and context pairs
    discriminator: bool
        Whether to create a dataset for the discriminator
    n_negative: int
        The number of random negative samples to add (using a random context)
    add_related: bool
        Whether to add the related contexts to the correct context. These are hard negative examples

    Returns
    -------
    pd.DataFrame
        The dataframe containing the prompts and completions, ready for fine-tuning
    """
    rows = []
    for i, row in df.iterrows():
        for q, a in zip(("1." + row.questions).split('\n'), ("1." + row.answers).split('\n')):
            if len(q) >10 and len(a) >10:
                if discriminator:
                    rows.append({"prompt":f"{row.context}\nQuestion: {q[2:].strip()}\n Related:", "completion":f" yes"})
                else:
                    rows.append({"prompt":f"{row.context}\nQuestion: {q[2:].strip()}\nAnswer:", "completion":f" {a[2:].strip()}"})

    for i, row in df.iterrows():
        for q in ("1." + row.questions).split('\n'):
            if len(q) >10:
                for j in range(n_negative + (2 if add_related else 0)):
                    random_context = ""
                    if j == 0 and add_related:
                        # add the related contexts based on originating from the same wikipedia page
                        subset = df[(df.title == row.title) & (df.context != row.context)]
                        
                        if len(subset) < 1:
                            continue
                        random_context = subset.sample(1).iloc[0].context
                    if j == 1 and add_related:
                        # add the related contexts based on the most similar contexts according to the search
                        for _, qe_row in df_contexts.iterrows():
                            if qe_row["question"][2:].strip() == q[2:].strip():
                                random_context = get_random_similar_contexts(qe_row["embedding"], row.context, max_rerank=10)
                    else:
                        while True:
                            # add random context, which isn't the correct context
                            random_context = df.sample(1).iloc[0].context
                            if random_context != row.context:
                                break
                    if discriminator:
                        rows.append({"prompt":f"{random_context}\nQuestion: {q[2:].strip()}\n Related:", "completion":f" no"})
                    else:
                        rows.append({"prompt":f"{random_context}\nQuestion: {q[2:].strip()}\nAnswer:", "completion":f" No appropriate context found to answer the question."})

    return pd.DataFrame(rows) 

We apply the same process of dataset creation for both the discriminator, and the Q&A answering model. We apply the process separately for the training and testing set, to ensure that the examples from the training set don't feature within the test set.

In [98]:
for name, is_disc in [('discriminator', True), ('qa', False)]:
    for train_test, dt in [('train', train_df), ('test', test_df)]:
        ft = create_fine_tuning_dataset(dt, discriminator=is_disc, n_negative=1, add_related=True)
        ft.to_json(f'olympics-data/{name}_{train_test}.jsonl', orient='records', lines=True)

In [106]:
for name, is_disc in [('discriminator', True), ('qa', False)]:
    ft = pd.read_json(f'olympics-data/{name}_test.jsonl', lines=True)
    print(f"Data set: {name}\n\n Example prompt: \n\n{ft['prompt'].iloc[0]}\n\n Example completion: \n\n{ft['completion'].iloc[0]}\n\n\n\n")

Data set: discriminator

 Example prompt: 

Boxing at the 2020 Summer Olympics – Men's lightweight
Competition format

Like all Olympic boxing events, the competition is a straight single-elimination tournament. The competition begins with a preliminary round, where the number of competitors is reduced to 16, and concludes with a final. As there are fewer than 32 boxers in the competition, a number of boxers will receive a bye through the preliminary round. Both semifinal losers are awarded bronze medals.
Bouts consist of three three-minute rounds with a one-minute break between rounds. A boxer may win by knockout or by points. Scoring is on the "10-point-must" system, with 5 judges scoring each round. Judges consider "number of blows landed on the target areas, domination of the bout, technique and tactical superiority and competitiveness." Each judge determines a winner for each round, who receives 10 points for the round, and assigns the round's loser a number of points between 7 an

We formatted the data according to the recommendations from the fine-tuning tool, which is available using
> openai tools fine_tunes.prepare_data -f qa_train.jsonl

We highly recommend that you use this tool, which suggests improvements in your data formatting for fine-tuning.


# The code below has not been started!

## 3.2 Submit the datasets for fine-tuning

In [ ]:
!openai api fine_tunes.create -t "olympics-data/discriminator_train.jsonl" -v "olympics-data/discriminator_test.jsonl" --batch_size 16  --compute_classification_metrics --classification_positive_class " yes" --model ada

In [ ]:
!openai api fine_tunes.create -t "olympics-data/qa_train.jsonl" -v "olympics-data/qa_test.jsonl" --batch_size 16

## 3.3 Using the fine-tuned models

We will now use the fine-tuned discriminator and the fine-tuned Q&A model. By requesting logprobs, we can see how certain the discriminator is in a `yes` vs `no` answer.

In [ ]:
ft_discriminator = "curie:ft-openai-internal-2021-08-23-23-58-57"
ft_qa = "curie:ft-openai-internal-2021-08-23-17-54-10"

def apply_ft_discriminator(context, question, discriminator_model):
    """
    Apply the fine tuned discriminator to a question, to assess whether it can be answered from the context.
    """
    prompt = f"{context}\nQuestion: {question}\n Related:"
    result = openai.Completion.create(model=discriminator_model, prompt=prompt, max_tokens=1, temperature=0, top_p=1, n=1, logprobs=2)
    return result['choices'][0]['logprobs']['top_logprobs']

apply_ft_discriminator('The first human-made object in space was the Soviet Union satellite Sputnik 1 on 4 October 1957.', 
                        'What was the first human-made object in space?', ft_discriminator)

We can see that the model can generalize well to different contexts and questions. 

In [ ]:
def apply_ft_qa_answer(context, question, answering_model):
    """
    Apply the fine tuned discriminator to a question
    """
    prompt = f"{context}\nQuestion: {question}\nAnswer:"
    result = openai.Completion.create(model=answering_model, prompt=prompt, max_tokens=30, temperature=0, top_p=1, n=1, stop=['.','\n'])
    return result['choices'][0]['text']

apply_ft_qa_answer('The first human-made object in space was the Soviet Union satellite Sputnik 1 on 4 October 1957.', 
                    'What was the first human-made object in space?', ft_qa)


We can see that the model can answer the question, when the context is appropriate.

In [ ]:
apply_ft_qa_answer('The first human-made object in space was the Soviet Union satellite Sputnik 1 on 4 October 1957.',
                    'What is impressive about the Soviet Union?', ft_qa)

In [ ]:
apply_ft_qa_answer('The first human-made object in space was the Soviet Union satellite Sputnik 1 on 4 October 1957.',
                    'How many cars were produced in the Soviet Union in 1970?', ft_qa)

We can see that the model knows when to answer the question, and when to say that insufficient context is present to answer the question.

We can also combine a discriminator and a base model, or a fine-tuned Q&A model. Discriminator can essentially serve as a decision whether the question can be answered given the context or not.

In [ ]:
def answer_question_conditionally(answering_model, discriminator_model, context, question, discriminator_logprob_yes_modifier=0):
    logprobs = apply_ft_discriminator(context, question, discriminator_model)
    yes_logprob = logprobs[' yes'] if ' yes' in logprobs else -100
    no_logprob = logprobs[' no'] if ' no' in logprobs else -100
    if yes_logprob + discriminator_logprob_yes_modifier < no_logprob:
        return " No appropriate context found to answer the question based on the discriminator."
    return apply_ft_qa_answer(context, question, answering_model)
answer_question_conditionally(ft_qa, ft_discriminator, 
                                "Crowdless games are a rare although not unheard-of occurrence in sports. \
                                 When they do occur, it is usually the result of events beyond the control \
                                 of the teams or fans, such as weather-related concerns, public health concerns, \
                                 or wider civil disturbances unrelated to the game. For instance, \
                                 the COVID-19 pandemic caused many sports leagues around the world \
                                 to be played behind closed doors.",
                                "Could weather cause a sport event to have no crowd?")

The above function illustrates how to potentially combine a discriminator and a fine-tuned Q&A model. This gives a more fine-grained control over how certain we want the model to be before it answers the question.

We'll now take a look on how answers endpoint works - combining search to retrieve the relevant context from a knowledge base, and then using the fine-tuned Q&A model to answer the question.

## 3.4 Answering the question based on a knowledge base
Finally we can use a logic similar to the [/answers](https://beta.openai.com/docs/api-reference/answers) endpoint, where we first search for the relevant context, and then ask a Q&A model to answer the question given that context. If you'd like to see the implementation details, check out the [`answers_with_ft.py`](answers_with_ft.py) file.

In [ ]:
from answers_with_ft import answer_question
answer_question(olympics_search_fileid, ft_qa, "Which country won the Women's football tournament at the 2020 Olympic games?")